<h3>Засоби підготовки та аналіз даних</h3>
<h2>Лабораторна робота №3</h2>
<h2>Наука про дані: обмін результатами та початковий аналіз</h2>
<h4>ФБ-33 Лозенко Павло</h4>

Мета роботи: ознайомитися з системою контролю версій GitHub, навчитися
створювати прості веб-додатки для обміну результатами досліджень із
використанням модуля spyre.<br>

Основні поняття: система контролю версій, репозитарій, інтерактивний веб-додаток.

<h3>Хід виконання роботи</h3>

Створення веб-додатоку із використанням модуля Streamlit:
1. Створіть dropdown список, який дозволить обрати часовий ряд VCI, TCI, VHI для
набору даних із лабораторної роботи 2;
2. Створіть dropdown список, який дозволить вибрати область, для якої буде
виконуватись аналіз;
3. Створіть slider, який дозволить зазначити інтервал тижнів, за які відбираються дані;
4. Створіть slider, який дозволить зазначити інтервал років, за які відбираються дані;
5. Створіть button для скидання всіх фільтрів і повернення до початкового стану даних
(відповідно інтерактивні елементи повинні мати початкові значення);
6. Створіть три вкладки для відображення таблиці з відфільтрованими даними,
відповідного до неї графіка та графіка порівняння даних по областях.
7. Перший графік повинен відображати відфільтровані дані (часові ряди за діапазон
років, що обмежені інтервалом тижнів). Другий графік має відображати порівняння
значень VCI, TCI або VHI (залежно від обраної опції у списку dropdown) для обраної
області з усіма іншими областями за вказаний часовий інтервал. Продумайте
вигляд цих графіків.
8. Створіть два checkbox для сортування даних за зростанням та спаданням значень
VCI, TCI або VHI (залежно від обраної опції у списку dropdown). Продумайте реакцію
програми, якщо увімкнені обидва чекбокси.
9. Інтерактивні елементи мають бути розміщений в одній колонці, а графіки з
таблицею — в іншій.
Робота з GitHub:
1. Зареєструйтесь на GitHub;
2. Створіть public репозиторій;
3. Додайте другу та третю лабораторні роботи в цей репозиторій. Кожна лабораторна
повинна бути в окремій директорії;
4. Репозиторій повинен мати заповнений файл README.MD та .gitignore, в якому
прописані винятки для Git: таблиці та інші дані, що використовуються.

In [ ]:
import os
import time
import __main__
from spyre import server 
from download_csv import *
from data_cleaning import *
from replace import *
import matplotlib.pyplot as plt
import seaborn as sns


if not hasattr(__main__, '__file__'):
    print("Error: Script must be run as a .py file, not in an interactive environment like Jupyter.")
    exit(1)

country = "UKR"
year_1 = 1982
year_2 = 2024
type_data = "Mean"

directory = r"Y:\labs\data\noaa_data"

PROVINCE_NAME_dict = {1: 'Вінницька', 2: 'Волинська', 3: 'Дніпропетровська', 4: 'Донецька', 5: 'Житомирська',
                      6: 'Закарпатська', 7: 'Запорізька', 8: 'Івано-Франківська', 9: 'Київська', 10: 'Кіровоградська',
                      11: 'Луганська', 12: 'Львівська', 13: 'Миколаївська', 14: 'Одеська', 15: 'Полтавська',
                      16: 'Рівенська', 17: 'Сумська', 18: 'Тернопільська', 19: 'Харківська', 20: 'Херсонська',
                      21: 'Хмельницька', 22: 'Черкаська', 23: 'Чернівецька', 24: 'Чернігівська', 25: 'Республіка Крим'}

def clean_directory(directory, retries=3, delay=2):
    for attempt in range(retries):
        try:
            os.makedirs(directory, exist_ok=True)
            print(f"[+] Directory ready: {directory}")
            return True
        except OSError as e:
            print(f"[-] Attempt {attempt + 1} failed: {e}")
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                print(f"[-] Error preparing directory {directory}: {e}")
                return False


if not os.path.exists("Y:"):
    print("Error: Y: drive is not accessible. Please ensure the drive is mapped.")
    exit(1)
if os.path.exists(directory) and not os.access(directory, os.W_OK):
    print(f"Error: Directory {directory} is not writable.")
    exit(1)

if not clean_directory(directory):
    print("Exiting due to directory creation failure.")
    exit(1)

download_csv(country, year_1, year_2, type_data, directory)
data_frames = read_data(directory)
data_frames_work = replace_function(data_frames)

class Web_Application(server.App):
    title = "National Oceanic and Atmospheric Administration🌍 NOAA data visualization" 
    inputs = [
        {
            "type": 'dropdown', 
            "label": 'NOAA data dropdown',
            "options": [{"label": 'Vegetation Condition Index (VCI)', "value": 'VCI'},
                        {"label": 'Temperature Condition Index (TCI)', "value": 'TCI'},
                        {"label": 'Vegetation Health (VHI)', "value": 'VHI'}],
            "key": 'index',
            "action_id": 'update_data'
        },
        {
            "type": 'dropdown',
            "label": 'Region',
            "options": [{"label": PROVINCE_NAME_dict[region], "value": region} for region in sorted(data_frames_work['PROVINCE_ID'].unique())],
            "key": 'region',
            "action_id": 'update_data'
        },
        {
            "type": 'text',
            "label": 'Range of weeks: (1 - 52)',
            "key": 'range_weeks',
            "value": '1 - 52',
            "action_id": 'simple_html_output'
        },
        {
            "type": 'text',
            "label": 'Range years',
            "key": 'range_year',
            "value": '1982-2024',
            "action_id": 'update_data'            
        }
    ]

    controls = [
        {
            "type": "button",
            "id": "update_data",
            "label": "Get information"
        }
    ]
    
    tabs = ["Table", "Graph"]
    
    outputs = [
        {
            "type": "plot",
            "id": "plot",
            "control_id": "update_data",
            "tab": "Graph",
            "on_page_load": True
        },
        {
            "type": "table",
            "id": "table_id",
            "control_id": "update_data",
            "tab": "Table",
            "on_page_load": True
        }
    ] 
    
    def getData(self, params):
        index = params["index"]
        region = int(params["region"])
        range_weeks = params["range_weeks"]
        range_year = params["range_year"]
        
        province_name = PROVINCE_NAME_dict.get(region, "")
             
        if isinstance(range_year, list):
            year_1, year_2 = range_year
            week_1, week_2 = range_weeks
        else:
            year_1, year_2 = map(int, range_year.split('-'))
            week_1, week_2 = map(int, range_weeks.split('-'))
        
        data_frame = data_frames_work[(data_frames_work["PROVINCE_ID"] == region) & 
                                    (data_frames_work["Week"].between(week_1, week_2)) &
                                    (data_frames_work["Year"].between(year_1, year_2))][["PROVINCE_ID", "Year", "Week", index]]
        
        data_frame.insert(1, "PROVINCE_NAME", province_name)
        
        return data_frame

    def getPlot(self, params):
        df = self.getData(params).drop(['PROVINCE_ID'], axis=1)
        index = params["index"]      

        figure, ax = plt.subplots(figsize=(17, 7)) 
        sns.lineplot(x='Week', y=index, data=df, label=index, marker='o', ax=ax, linestyle='-', color='black')

        ax.set_title(f"Noaa data visualization {index} for {country}: region: {df.iloc[0]['PROVINCE_NAME']} ", fontsize=18, color='black', weight='bold')  
        ax.set_xlabel("Week", fontsize=15, color='black', weight='bold')  
        ax.set_ylabel(index, fontsize=15, color='black', weight='bold')   

        ax.grid(True, linestyle='--', alpha=0.95)
        ax.legend(fontsize=15)
        ax.tick_params(axis='both', which='major', labelsize=10, colors='black')  

        ax.set_xlim(df['Week'].min() - 1, df['Week'].max() + 1)    
        ax.set_ylim(df[index].min() * 0.9, df[index].max() * 1.1) 

        plt.xticks(rotation=45)
        plt.yticks(rotation=45)

        ax.annotate('Start', xy=(df['Week'].min(), df[index].min()), xytext=(df['Week'].min(), df[index].min() - 20),
                    arrowprops=dict(facecolor='red', shrink=0.05), fontsize=10, color='black', horizontalalignment='center')

        ax.annotate('End', xy=(df['Week'].max(), df[index].min()), xytext=(df['Week'].max(), df[index].min() - 20),
                    arrowprops=dict(facecolor='red', shrink=0.05), fontsize=10, color='black', horizontalalignment='center')

        return figure

if __name__ == "__main__":
    app = Web_Application()
    app.launch(port=9995)